In [ ]:
# load things
%matplotlib inline

# the following automatically loads the following modules:
# 
# matplotlib
# plt        = matplotlib.plt
# np         = numpy
# cartopy
# ccrs       = cartopy.ccrs
#
# with some default stylings (see file itself). See file for commands

from orca_plotting_commands import *
from matplotlib_colorbar import *

# modules for loading data
import netCDF4
# need iris if wanting to use cartopy_command
import iris
import iris.analysis.cartography

# for editing
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

from pyCDFTOOLS.cdfspeed import *

In [ ]:
# load data and do relevant processing on the native grid

data_dir = "/home/julian/data/NEMO_data/eORCA1-LIM3/default/"
fileU = "eORCA1_10y_06510101_06601231_grid_U.nc"
fileV = "eORCA1_10y_06510101_06601231_grid_V.nc"

# for putting extra options in
#   -- kt       = number for using a specified time entry
#   -- kz       = number for using a specified vertical level/layer
#   -- lprint   = True   for printing out variable names in netcdf file
#   -- lperio   = True   for E-W periodicity
kwargs = {"lprint" : False,
          "lperio" : False}

lonT, latT, absu, opt_dic = cdfspeed(data_dir, fileU, "uo", fileV, "vo", **kwargs)

In [ ]:
# load in iris and defined some generic things
iris.FUTURE.netcdf_promote = True

figure_save = False
save_filename = "./ORCA1_testing.png"
axis_vec = [-180, 180, -75, 90] # for north Atlantic # [-90, 20, 20, 85] 

pcarree = ccrs.PlateCarree()

target_proj = ccrs.PlateCarree()
#target_proj = ccrs.Orthographic(central_longitude=0, central_latitude=-90) # SO focus
#target_proj = ccrs.Orthographic(central_longitude=-50, central_latitude=30) # N Atlantic Focus
#target_proj = ccrs.Mercator()
#target_proj = ccrs.Robinson()
#target_proj = ccrs.Mollweide()

lat = iris.coords.AuxCoord(latT, standard_name = "latitude", units = "degrees")
lon = iris.coords.AuxCoord(lonT, standard_name = "longitude", units = "degrees")
data_cube = iris.cube.Cube(absu, 
                           long_name = "mod_u", 
                           units = "m s-1",
                           aux_coords_and_dims = [(lat, (0, 1)), (lon, (0,1))])
data_proj, extent = iris.analysis.cartography.project(data_cube[:, :], pcarree, nx = 600, ny = 300)
x = data_proj.coord('projection_x_coordinate').points
y = data_proj.coord('projection_y_coordinate').points
plot_data = data_proj.data

In [ ]:
# touch up the data and set levels
# "mask" the data by setting the land point data to be the minimum of the cmin
cmin = plot_data.min()
cmax = np.ceil(plot_data.max())
plot_data[(plot_data == 0)] = cmin
cmin = np.floor(cmin)

if type(target_proj).__name__ in ["Orthographic"]:
    if figure_save:
        fig = plt.figure(figsize=(5, 5), dpi = 300)
    else:
        fig = plt.figure(figsize=(5, 5))
else:
    if figure_save:
        fig = plt.figure(figsize=(10, 7), dpi = 300)
    else:
        fig = plt.figure(figsize=(10, 7))
    
misc_format = {"levels" : np.linspace(0, 0.4, 51),
               "extend" : "both",
               "cmap"   : "Blues_r"}

ax, mesh = cartopy_contourf(x, y, plot_data, target_proj, **misc_format)

#misc_format = {"vmin" : 0, "vmax" : 0.5,
#               "cmap"   : "Blues_r"}
#ax, mesh = cartopy_pcolormesh(x, y, plot_data, target_proj, **misc_format)

# set axes, title and add gridlines

ax.gridlines(linewidth = 1, linestyle = '--')

if type(target_proj).__name__ in ["PlateCarree", "Mercator"]:
    gl = ax.gridlines(crs=ccrs.PlateCarree(),
                  draw_labels = True, linewidth = 1, linestyle = '--')
    gl.xlabels_top = False
    gl.ylabels_right = False
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER

    ax.text(-0.05, 0.5, r'Lat $\left( {}^\circ \right)$', 
            va='bottom', ha='center',
            rotation=90, rotation_mode='anchor',
            transform=ax.transAxes)
    ax.text(0.5, -0.1, r'Lon $\left( {}^\circ \right)$', 
            va='bottom', ha='center',
            rotation='horizontal', rotation_mode='anchor',
            transform=ax.transAxes)
    
    ax.set_extent(axis_vec, crs = ccrs.PlateCarree())

# add colorbar
divider = make_axes_locatable(ax)
if type(target_proj).__name__ in ["Orthographic", "Robinson", "Mollweide"]:
    ax_cb = divider.new_vertical(size="2%", pad=0.2, pack_start=True, axes_class=plt.Axes)
    fig.add_axes(ax_cb)
    cb = plt.colorbar(mesh, cax=ax_cb, orientation = "horizontal")
    plt.xlabel(r'$\mathrm{m}\ \mathrm{s}^{-1}$') # label on colourbar
else:
    ax_cb = divider.new_horizontal(size="1%", pad=0.2, axes_class=plt.Axes)
    fig.add_axes(ax_cb)
    cb = plt.colorbar(mesh, cax=ax_cb, orientation = "vertical")
    plt.title(r'$\mathrm{m}\ \mathrm{s}^{-1}$') # title on colourbar

if figure_save:
    fig.savefig(save_filename, bbox_inches = "tight")
    plt.close(fig)
    
print("--- Done! ---")